In [1]:
from covidcf.evaluation.base import load_data, prepreprocess_data
from covidcf.config.base import load_config
from covidcf.config.dataset import DatasetTarget
import os

In [2]:
os.chdir('..')

In [9]:
df = load_data('ictcf')
df

Input                                                                \
      Clinical                                                                 
     Value_5NT Value_AFU Value_ALB Value_ALG Value_ALP Value_ALT Value_APOA1   
0          0.6       NaN    -0.015      -0.6  0.109091  1.657143         NaN   
1          0.9       NaN     0.295       0.2 -0.036364  1.285714         NaN   
2          NaN       NaN    -0.250      -1.2  0.036364  1.542857         NaN   
3          0.8       NaN    -0.165      -0.6  0.236364  0.857143         NaN   
4          NaN       NaN       NaN       NaN       NaN       NaN         NaN   
...        ...       ...       ...       ...       ...       ...         ...   
1516       0.9       NaN    -0.045      -0.8  0.172727  2.600000         NaN   
1517       NaN       NaN       NaN       NaN       NaN       NaN         NaN   
1518       NaN       NaN       NaN       NaN       NaN       NaN         NaN   
1519       NaN       NaN       NaN       NaN       NaN       NaN         NaN   
1520       NaN       NaN       NaN       NaN       NaN       NaN         NaN   

                                      ...                                \
                                      ...         Visual                  
     Value_APOB Value_APTT Value_ASO  ... vis_feature_51 vis_feature_52   
0           NaN   0.677419       NaN  ...       1.407101       0.086921   
1           NaN   1.012903       NaN  ...       5.079040       3.516342   
2           NaN        NaN       NaN  ...       0.061765      -0.239709   
3           NaN   0.696774       NaN  ...       0.878605       0.145058   
4           NaN        NaN       NaN  ...      -0.146057      -0.608026   
...         ...        ...       ...  ...            ...            ...   
1516        NaN   0.935484       NaN  ...       2.507623       0.919673   
1517        NaN        NaN       NaN  ...      -0.590748      -0.850672   
1518        NaN        NaN       NaN  ...      -0.496094      -0.825329   
1519        NaN        NaN       NaN  ...       0.111390      -0.201476   
1520        NaN        NaN       NaN  ...       1.792377       1.302997   

                                                                  \
                                                                   
     vis_feature_53 vis_feature_54 vis_feature_55 vis_feature_56   
0          0.072483       0.069505       0.051657      -0.409348   
1          3.404648       3.442693       3.337378       1.271466   
2         -0.255159      -0.260668      -0.277846      -0.393225   
3          0.107863       0.133092       0.092496      -0.333626   
4         -0.617320      -0.617773      -0.630355      -0.611461   
...             ...            ...            ...            ...   
1516       0.772426       0.912312       0.760904      -0.141144   
1517      -0.832764      -0.867031      -0.852193      -0.569649   
1518      -0.808753      -0.834399      -0.821405      -0.645694   
1519      -0.209360      -0.209015      -0.221224      -0.407458   
1520       7.887997       1.169276       0.061628      -4.302420   

                                                    Meta  
                                                  cohort  
     vis_feature_57 vis_feature_58 vis_feature_59         
0         -0.414904      -0.435154      -0.488452     c1  
1          1.221740       1.255130       1.191330     c1  
2         -0.406874      -0.414642      -0.454006     c1  
3         -0.353214      -0.352350      -0.419885     c1  
4         -0.623873      -0.624798      -0.681182     c1  
...             ...            ...            ...    ...  
1516      -0.196170      -0.162558      -0.266687     c2  
1517      -0.572261      -0.597436      -0.649602     c2  
1518      -0.645272      -0.667305      -0.714351     c2  
1519      -0.420308      -0.431136      -0.493907     c2  
1520      -4.605498      -4.470306      -0.518231     c2  

[1521 rows x 193 columns]

In [10]:
config = load_config('experiments/within_dataset/ictcf/combined/gbdt.yaml')

In [11]:
config.dataset.target = DatasetTarget.PCR

In [12]:
df = prepreprocess_data(df, config)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [13]:
df, X, y = df

In [14]:
df.to_('df.csv')
X.to_csv('X.csv')
y.to_csv('y.csv')

In [18]:
y.isna().any()

False

In [26]:
for col in X.columns:
    if X[col].dtype.name == 'category':
        X[col].cat.add_categories('?', inplace=True)
        X.loc[X[col].isna(), col] = '?'


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [43]:
import arff
arff.dump('X_train.arff', pd.concat([X[df.Meta.hospital == 'Union'], y[df.Meta.hospital == 'Union'] == 'Positive'], axis=1).values, relation='X_train', names=list(X.columns.str.replace(' ', '_')) + ['pcr'])

In [44]:
arff.dump('X_test.arff', pd.concat([X[df.Meta.hospital != 'Union'], y[df.Meta.hospital != 'Union'] == 'Positive'], axis=1).values, relation='X_train', names=list(X.columns.str.replace(' ', '_')) + ['pcr'])

In [33]:
pd.concat([X.fillna('?'), y], axis=1).values

array([[-0.014999999999999857, -0.6000000000000001, 0.10909090909090909,
        ..., -0.4351544, -0.48845235, 'Positive'],
       [0.29499999999999993, 0.20000000000000018, -0.03636363636363636,
        ..., 1.25513, 1.19133, 'Positive'],
       [-0.25, -1.2, 0.03636363636363636, ..., -0.4146418, -0.454006,
        'Positive'],
       ...,
       ['?', '?', '?', ..., -0.667305, -0.714351, 'Positive'],
       ['?', '?', '?', ..., -0.4311356, -0.49390742, 'Positive'],
       ['?', '?', '?', ..., -4.47030604, -0.518230915, 'Positive']],
      dtype=object)

In [9]:
X_train, y_train = X[df.Meta.part != 'test'], y[df.Meta.part != 'test']
X_test, y_test, = X[df.Meta.part == 'test'], y[df.Meta.part == 'test']

In [16]:
from bartpy.sklearnmodel import SklearnModel

In [17]:
model = SklearnModel(n_jobs=1)
model.fit(X_train, y_train)

  0%|                                                                   | 0/200 [00:00<?, ?it/s]

Starting burn


divide by zero encountered in log
invalid value encountered in double_scalars
divide by zero encountered in true_divide
invalid value encountered in double_scalars
  0%|▎                                                          | 1/200 [00:00<01:51,  1.78it/s]invalid value encountered in sqrt
divide by zero encountered in double_scalars
 36%|█████████████████████▏                                    | 73/200 [00:15<00:27,  4.65it/s]divide by zero encountered in double_scalars
invalid value encountered in double_scalars
  0%|                                                                   | 0/200 [00:00<?, ?it/s]

Starting sampling


  0%|                                                                   | 0/200 [00:00<?, ?it/s]

...

Starting burn


  0%|                                                                   | 0/200 [00:00<?, ?it/s]

Starting sampling


100%|█████████████████████████████████████████████████████████| 200/200 [00:44<00:00,  4.52it/s]


SklearnModel(n_jobs=1)

In [18]:
model.score(X_test, y_test)

-0.04465648854961857

In [11]:
from lightgbm.sklearn import LGBMRegressor

In [21]:
reg = LGBMRegressor(n_jobs=-1)
reg.fit(X_train, y_train)

LGBMRegressor()

In [23]:
from sklearn.metrics import roc_auc_score
import numpy as np
def score_corads(y_true, y_pred):
    y_true = y_true >= 3
    return roc_auc_score(y_true, np.clip(y_pred, 0, 4)/4)

In [28]:
prepreprocess_data(X)

TypeError: prepreprocess_data() missing 1 required positional argument: 'config'

In [19]:
model_preds = model.predict(X_test)

In [27]:
model_preds

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [20]:
from sklearn.metrics import r2_score
r2_score(y_test, model_preds)

-0.04465648854961857

In [25]:
score_corads(y_test, model_preds)

0.5

In [10]:
!pip install git+https://github.com/JakeColtman/bartpy.git --upgrade

  Cloning https://github.com/JakeColtman/bartpy.git to c:\users\roel\appdata\local\temp\pip-req-build-hsn38px2
  Created wheel for bartpy: filename=bartpy-0.0.2-py3-none-any.whl size=53446 sha256=63335b9f45de78f751d17c7ed8c6395fbe62411a87d43611116bbdc17bb587e6
  Stored in directory: C:\Users\Roel\AppData\Local\Temp\pip-ephem-wheel-cache-3sywe7no\wheels\d9\23\41\d13943255fc60cbf0a088b80529b2f060fb36b00b229ba518e
Successfully built bartpy


  Running command git clone -q https://github.com/JakeColtman/bartpy.git 'C:\Users\Roel\AppData\Local\Temp\pip-req-build-hsn38px2'
You should consider upgrading via the 'c:\users\roel\.virtualenvs\diagcovid-ckc2berq\scripts\python.exe -m pip install --upgrade pip' command.
